In [2]:
import os
from os.path import join
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
import xgboost; print(xgboost.__version__)
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor, XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from scipy.stats import uniform, randint
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

1.3.1


In [3]:
path_main = "/Users/jurriaan/ML/kaggle_ObservingDarkWorlds"
path_data = join(path_main, "DarkWorlds")
                 

In [4]:
halos_train_df = pd.read_csv(join(path_data, "Training_halos.csv"))
halos_train_df

,SkyId,numberHalos,x_ref,y_ref,halo_x1,halo_y1,halo_x2,halo_y2,halo_x3,halo_y3
0,Sky1,1,1086.80,1114.61,1086.80,1114.61,0.00,0.00,0.00,0.00
1,Sky2,1,3477.71,1907.33,3477.71,1907.33,0.00,0.00,0.00,0.00
2,Sky3,1,2315.78,1081.95,2315.78,1081.95,0.00,0.00,0.00,0.00
3,Sky4,1,1408.61,1685.86,1408.61,1685.86,0.00,0.00,0.00,0.00
4,Sky5,1,3091.65,2001.08,3091.65,2001.08,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...
295,Sky296,3,832.11,1287.86,760.42,1270.57,2493.50,1915.87,1700.29,1303.56
296,Sky297,3,2790.06,340.14,2712.63,345.29,3910.23,472.60,3438.33,152.13
297,Sky298,3,3555.68,3729.10,3837.68,4173.71,4166.00,3076.44,1773.77,1157.65
298,Sky299,3,2413.48,2036.56,2906.06,1988.74,1708.82,1413.22,561.41,2569.31


In [5]:
halos_train_df["x_ref"].max()

4169.9

In [4]:
halos_train_df["numberHalos"].value_counts()

3    100
2    100
1    100
Name: numberHalos, dtype: int64

In [5]:
skies_train_df_1 = pd.read_csv(join(path_data, "Train_Skies", "Train_Skies", "Training_Sky1.csv"))
skies_train_df_1

,GalaxyID,x,y,e1,e2
0,Galaxy1,700.87,89.98,-0.050569,-0.301238
1,Galaxy2,570.74,3528.23,-0.248973,0.052048
2,Galaxy3,579.66,1381.65,0.483764,0.060891
3,Galaxy4,1595.64,1226.96,-0.043464,0.207885
4,Galaxy5,444.24,1567.29,-0.102021,-0.207315
...,...,...,...,...,...
343,Galaxy344,3548.83,306.39,0.194440,-0.364194
344,Galaxy345,2957.20,966.09,-0.323762,0.443509
345,Galaxy346,3555.70,722.22,-0.249272,-0.380333
346,Galaxy347,1845.34,3458.47,-0.460082,-0.277290


### benchmarks

In [6]:
random_bm_df = pd.read_csv(join(path_data, "Random_Benchmark.csv"))
grid_bm_df = pd.read_csv(join(path_data, "Gridded_Signal_benchmark.csv"))
ml_bm_df = pd.read_csv(join(path_data, "Maximum_likelihood_Benchmark.csv"))

In [7]:
random_bm_df

,SkyId,pred_x1,pred_y1,pred_x2,pred_y2,pred_x3,pred_y3
0,Sky1,3203.725556,4114.378609,0.000000,0.000000,0.000000,0.000000
1,Sky2,1731.525703,3671.572600,0.000000,0.000000,0.000000,0.000000
2,Sky3,2556.242919,1832.568154,0.000000,0.000000,0.000000,0.000000
3,Sky4,2782.783073,2380.429869,0.000000,0.000000,0.000000,0.000000
4,Sky5,1814.926386,3685.402309,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
115,Sky116,1065.694247,4070.213861,991.340812,4072.033602,924.474433,1483.392023
116,Sky117,3752.513536,779.995851,3743.606071,2947.413352,3551.396169,2.185555
117,Sky118,3648.657944,4072.412657,1641.422991,1273.284845,2744.516341,313.035378
118,Sky119,2348.944574,3252.233133,418.849328,1810.638708,1934.962363,1375.964364


### couple galaxies to halos

In [72]:
skies_train_df_1["x"]*skies_train_df_1["x"]

0      4.912188e+05
1      3.257441e+05
2      3.360057e+05
3      2.546067e+06
4      1.973492e+05
           ...     
343    1.259419e+07
344    8.745032e+06
345    1.264300e+07
346    3.405280e+06
347    2.169847e+06
Name: x, Length: 348, dtype: float64

In [6]:
list_r = []
list_et = []


for i in d_skies_train_df:
    skies_train_df = d_skies_train_df[i]
    
    halo_x1 = halos_train_df.loc[i, "halo_x1"].squeeze()
    halo_y1 = halos_train_df.loc[i, "halo_y1"].squeeze()
    
    x = skies_train_df["x"].values
    y = skies_train_df["y"].values
    e1 = skies_train_df["e1"].values
    e2 = skies_train_df["e2"].values

    r_from_halo = np.sqrt((x-halo_x1)**2+(y-halo_y1)**2)
    angle_wrt_centre = np.arctan2((y-halo_y1)/(x-halo_x1))
    e1_force = e1*np.cos(2.*angle_wrt_centre)
    e2_force = e2*np.sin(2.*angle_wrt_centre)
    et = -1*(e1_force + e2_force)
    
    list_r.append(r_from_halo)
    list_et.append(et)
    

    if i > 10: 
        break

list_r_fl = [r for l in list_r for r in l]
list_et_fl = [r for l in list_et for r in l]

NameError: name 'd_skies_train_df' is not defined

In [158]:
max_len = 0
for index, row in halos_train_df.copy().iterrows():    
    row_large = row.values
    skies_train_df_i = pd.read_csv(join(path_data, "Train_Skies", "Train_Skies", f"Training_Sky{index + 1}.csv"))
    max_len = max(max_len, len(skies_train_df_i))
    
super_list = []
for index, row in halos_train_df.copy().iterrows():    
    row_large = row.values
    skies_train_df_i = pd.read_csv(join(path_data, "Train_Skies", "Train_Skies", f"Training_Sky{index + 1}.csv"))
    for idx, r in skies_train_df_i[["e1", "e2"]].iterrows():
        e1 = r["e1"]
        e2 = r["e2"]
        row_large = np.append(row_large, np.array([e1, e2]))
    
    super_list.append(row_large)

In [159]:
max([len(x) for x in super_list]), (min([len(x) for x in super_list]))

(1482, 646)

In [143]:
len(super_list)

300

In [160]:
df_super_e = pd.DataFrame(super_list)

In [161]:
df_super_e.rename(columns={0:"SkyId", 1:"numberHalos", 2:"x_ref", 3:"y_ref", 4:"halo_x1",
                         5:"halo_y1", 6:"halo_x2", 7:"halo_y2", 8:"halo_x3", 
                        9:"halo_y3"}, inplace=True)

In [162]:
df_super_e.head()

,SkyId,numberHalos,x_ref,y_ref,halo_x1,halo_y1,halo_x2,halo_y2,halo_x3,halo_y3,...,1472,1473,1474,1475,1476,1477,1478,1479,1480,1481
0,Sky1,1,1086.80,1114.61,1086.80,1114.61,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sky2,1,3477.71,1907.33,3477.71,1907.33,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sky3,1,2315.78,1081.95,2315.78,1081.95,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sky4,1,1408.61,1685.86,1408.61,1685.86,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sky5,1,3091.65,2001.08,3091.65,2001.08,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
sum((df_super3.iloc[3,11:] > 10))

1059

In [111]:
# e1,e2 starts at 1070, xy starts at 10

In [140]:
df_super3.iloc[3,5:15]

halo_y1        1685.86
halo_x2              0
halo_y2              0
halo_x3              0
halo_y3              0
10         1.38971e+06
11         2.06718e+06
12            0.064384
13           -0.005815
14              341745
Name: 3, dtype: object

In [116]:
df_super2.iloc[3,1065:1075]

1065       0.013682
1066    1.03531e+06
1067         370005
1068      -0.120899
1069       0.303557
1070    4.47136e+06
1071    1.27822e+07
1072      -0.155753
1073       0.045854
1074    4.22081e+06
Name: 3, dtype: object

In [163]:
df_super_e.to_csv("df_super_e.csv", index=False)

### explore

In [165]:
# relation # halos and coors/ellip visible?
df_super3.groupby("numberHalos").mean().iloc[:,8:].mean(axis=1)

numberHalos
1    5.912777e+06
2    5.888880e+06
3    5.853737e+06
dtype: float64

In [164]:
# relation # halos and coors/ellip visible?
df_super_e.groupby("numberHalos").mean().iloc[:,8:].mean(axis=1)

numberHalos
1    0.000903
2    0.000463
3   -0.001399
dtype: float64

In [151]:
df_super3.groupby("numberHalos").std().mean(axis=1)

numberHalos
1    5.197528e+06
2    5.177629e+06
3    5.179820e+06
dtype: float64

In [166]:
df_super_e.groupby("numberHalos").std().mean(axis=1)

numberHalos
1    3.401764
2    4.950773
3    6.724041
dtype: float64

In [167]:
df_super2

,SkyId,numberHalos,x_ref,y_ref,halo_x1,halo_y1,halo_x2,halo_y2,halo_x3,halo_y3,...,2944,2945,2946,2947,2948,2949,2950,2951,2952,2953
0,Sky1,1,1086.80,1114.61,1086.80,1114.61,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sky2,1,3477.71,1907.33,3477.71,1907.33,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sky3,1,2315.78,1081.95,2315.78,1081.95,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sky4,1,1408.61,1685.86,1408.61,1685.86,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sky5,1,3091.65,2001.08,3091.65,2001.08,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,Sky296,3,832.11,1287.86,760.42,1270.57,2493.50,1915.87,1700.29,1303.56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,Sky297,3,2790.06,340.14,2712.63,345.29,3910.23,472.60,3438.33,152.13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,Sky298,3,3555.68,3729.10,3837.68,4173.71,4166.00,3076.44,1773.77,1157.65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
298,Sky299,3,2413.48,2036.56,2906.06,1988.74,1708.82,1413.22,561.41,2569.31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### load data from csv

In [7]:
df_super_e = pd.read_csv("df_super_e.csv")
df_super_e.head()

,SkyId,numberHalos,x_ref,y_ref,halo_x1,halo_y1,halo_x2,halo_y2,halo_x3,halo_y3,...,1472,1473,1474,1475,1476,1477,1478,1479,1480,1481
0,Sky1,1,1086.80,1114.61,1086.80,1114.61,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sky2,1,3477.71,1907.33,3477.71,1907.33,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sky3,1,2315.78,1081.95,2315.78,1081.95,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sky4,1,1408.61,1685.86,1408.61,1685.86,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sky5,1,3091.65,2001.08,3091.65,2001.08,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_super_xy = pd.read_csv("df_super_xy.csv")
df_super_xy.head()

,SkyId,numberHalos,x_ref,y_ref,halo_x1,halo_y1,halo_x2,halo_y2,halo_x3,halo_y3,...,1472,1473,1474,1475,1476,1477,1478,1479,1480,1481
0,Sky1,1,1086.80,1114.61,1086.80,1114.61,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sky2,1,3477.71,1907.33,3477.71,1907.33,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sky3,1,2315.78,1081.95,2315.78,1081.95,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sky4,1,1408.61,1685.86,1408.61,1685.86,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sky5,1,3091.65,2001.08,3091.65,2001.08,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Prepare for ML

In [10]:
df_X_xy = df_super_xy.iloc[:,10:]
df_X_xy.head()

,10,11,12,13,14,15,16,17,18,19,...,1472,1473,1474,1475,1476,1477,1478,1479,1480,1481
0,4.912188e+05,8.096400e+03,3.257441e+05,1.244841e+07,3.360057e+05,1.908957e+06,2.546067e+06,1.505431e+06,1.973492e+05,2.456398e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.948526e+06,5.362235e+06,1.051404e+06,1.407759e+06,3.775857e+05,3.714408e+06,1.696534e+07,5.946261e+05,1.349695e+07,1.213909e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.646804e+04,2.560192e+06,5.163256e+06,2.916432e+05,1.262836e+07,7.277639e+06,1.867350e+06,1.689457e+07,1.679680e+07,1.165553e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.389711e+06,2.067183e+06,3.417455e+05,3.618251e+06,1.267040e+07,1.125200e+07,1.066717e+06,1.169982e+05,9.410940e+05,9.507417e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7.009256e+06,8.612211e+05,5.151493e+06,2.048391e+06,5.846144e+06,1.579823e+06,2.316423e+06,2.221084e+06,4.012810e+06,1.397339e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
scaler = StandardScaler()
X_xy_scale = scaler.fit_transform(df_X_xy)

In [13]:
X_xy_scale.shape

(300, 1472)

In [14]:
df_X_e = df_super_e.iloc[:,10:].values
df_X_e

array([[-0.050569, -0.301238, -0.248973, ...,       nan,       nan,
              nan],
       [-0.39345 , -0.206903,  0.201002, ...,       nan,       nan,
              nan],
       [ 0.114664, -0.190326,  0.623555, ...,       nan,       nan,
              nan],
       ...,
       [-0.043903, -0.03822 ,  0.077709, ...,       nan,       nan,
              nan],
       [-0.32554 ,  0.096819,  0.165547, ...,       nan,       nan,
              nan],
       [-0.194881,  0.045802, -0.235309, ...,       nan,       nan,
              nan]])

In [15]:
data_X = np.concatenate([X_xy_scale, df_X_e], axis=1)
data_X

array([[-0.94789466, -1.13080535, -1.03128686, ...,         nan,
                nan,         nan],
       [ 0.10243919, -0.11237472, -0.89501436, ...,         nan,
                nan,         nan],
       [-1.03734233, -0.64536174, -0.12284503, ...,         nan,
                nan,         nan],
       ...,
       [-1.01416876, -1.08308013,  0.86027447, ...,         nan,
                nan,         nan],
       [-0.71061469,  1.95232968, -0.93287243, ...,         nan,
                nan,         nan],
       [-0.53097911, -0.93167458, -0.88276042, ...,         nan,
                nan,         nan]])

In [16]:
data_X.shape

(300, 2944)

In [17]:
se_Y = df_super_e["numberHalos"].values

In [18]:
se_Y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [20]:
X_train, X_test, y_train, y_test = train_test_split(data_X, se_Y, test_size=0.2)

### ML 1, predictions n halos

In [38]:
model = XGBRegressor(objective="multi:softprob", random_state=42, eval_metric="mlogloss")

In [42]:
set(y_train)

{1, 2, 3}

In [39]:
scores = cross_val_score(model, X_train, y_train)

/Users/jurriaan/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
xgboost.core.XGBoostError: value 0 for Parameter num_class should be greater equal to 1
num_class: Number of output class in the multi-class classification.

  FitFailedWarning)


In [37]:
scores

array([-0.0949423 ,  0.07878013, -0.19027256, -0.14266256, -0.17981539])

In [35]:
scores

array([-0.69526935, -0.61254724, -0.90458648, -0.63828416, -0.80651443])

In [44]:
kfold = KFold(n_splits=5)
scores2 = cross_val_score(model, data_X, se_Y, cv=kfold)

In [45]:
scores2

array([nan, nan, nan, nan, nan])

In [53]:
xgb_model = XGBClassifier(objective="multi:softprob", random_state=42)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)
print(confusion_matrix(y_test, y_pred))

[20:57:47] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[[ 8  6  8]
 [ 5  9  6]
 [ 2  6 10]]


In [54]:
y_test

array([2, 2, 3, 1, 2, 2, 1, 3, 3, 3, 1, 1, 1, 3, 2, 1, 2, 2, 3, 1, 2, 3,
       1, 1, 3, 2, 3, 3, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 3, 2, 1, 2, 2, 2,
       3, 1, 2, 2, 3, 3, 3, 1, 3, 1, 3, 1, 2, 2, 1, 3])

In [57]:
uniform(0.7, 0.3)

In [65]:
def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [64]:
params = {
    "colsample_bytree": uniform(0.7, 0.3),
    "gamma": uniform(0, 0.5),
    "learning_rate": uniform(0.03, 0.3), # default 0.1 
    "max_depth": randint(2, 6), # default 3
    "n_estimators": randint(100, 150), # default 100
    "subsample": uniform(0.6, 0.4)
}

search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state=42, 
                            n_iter=10, cv=3, verbose=1, n_jobs=1, return_train_score=True)

search.fit(data_X, se_Y)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[21:14:05] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/jurriaan/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:14:09] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:14:14] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:14:18] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:14:22] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the object

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  2.6min finished


[21:16:41] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


NameError: name 'report_best_scores' is not defined

In [67]:
search.cv_results_

{'mean_fit_time': array([4.08502173, 4.35561005, 4.58924158, 5.62365611, 6.83256205,
        6.18910805, 5.15712611, 3.79552873, 6.76417931, 4.17321142]),
 'std_fit_time': array([0.00456399, 0.33990325, 0.40140416, 0.38579183, 0.33183558,
        0.05081127, 0.1099111 , 0.03767505, 0.0909294 , 0.06077247]),
 'mean_score_time': array([0.00447122, 0.00640289, 0.00484101, 0.00504891, 0.00477441,
        0.00685628, 0.0043846 , 0.00435758, 0.00507593, 0.00457462]),
 'std_score_time': array([0.00041555, 0.00148599, 0.00086913, 0.00077101, 0.00056478,
        0.00353076, 0.00039062, 0.00039091, 0.00094465, 0.0005017 ]),
 'param_colsample_bytree': masked_array(data=[0.8123620356542087, 0.7467983561008608,
                    0.7061753482887407, 0.7550213529560301,
                    0.8835558684167137, 0.935552788417904,
                    0.9040922615763338, 0.7913841307520112,
                    0.9499584735208493, 0.8275467623473733],
              mask=[False, False, False, False, Fals

In [66]:
report_best_scores(search.cv_results_, 1)

Model with rank: 1
Mean validation score: 0.430 (std: 0.054)
Parameters: {'colsample_bytree': 0.7550213529560301, 'gamma': 0.15212112147976886, 'learning_rate': 0.18742692948967135, 'max_depth': 5, 'n_estimators': 124, 'subsample': 0.7164916560792167}



In [62]:
0.00054600*25000*100

1365.0

In [63]:
0.032*25000

800.0

### ml2 predictions positions x,y

In [83]:
from sklearn.datasets import make_multilabel_classification
X_test, Y_test = make_multilabel_classification(n_classes=5, n_labels=1,
                                                  allow_unlabeled=True,
                                                random_state=1)

In [84]:
X_test

array([[6., 0., 3., ..., 2., 2., 4.],
       [4., 0., 3., ..., 4., 1., 5.],
       [6., 5., 0., ..., 4., 0., 1.],
       ...,
       [0., 3., 0., ..., 4., 4., 3.],
       [0., 0., 3., ..., 1., 1., 3.],
       [0., 0., 7., ..., 2., 1., 0.]])

In [85]:
Y_test

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [1, 1, 0, 1, 0],
       [0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 1, 0, 0, 1],
       [1, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [1, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0],
       [1, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 1, 0, 1, 1],
       [0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0],
       [1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1],
       [1, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [1, 1, 0, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [1, 1, 0, 0, 1],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [1, 0, 0,